In [ ]:
import numpy as np
import minisom as som 
from pysomvis import PySOMVis

#init synthetic data
x = np.linspace(0,15,500)
y = np.sin(x)
y += np.random.normal(0,.2,len(y))
synth = np.stack((x,y,np.arange(0,len(x)))).T

#train with miniSOM library
s = som.MiniSom(x=20, y=2, input_len=synth.shape[1], sigma=1.0, learning_rate=0.8, random_seed=7)
s.train_random(synth, 100, verbose=False)

#visualization
pysomviz = PySOMVis(weights=s._weights, input_data=synth)
pysomviz._mainview

In [ ]:
import pandas as pd
import numpy as np
from pysomvis import PySOMVis
import minisom as som
import datetime
from sklearn import datasets, preprocessing

data = pd.read_excel('temps.xlsx', index_col=0)
data = data.reset_index()

def calc(row):
    return datetime.datetime(int(row[0]), int(row[1]), int(row[2]))

data['time'] = data.apply(calc, axis=1)
data.drop(data.columns[[0, 1, 2, 5, 7, 9]], axis=1, inplace=True)

data['RH2M'] = data.groupby(['RH2M']).apply(lambda x: x[['RH2M']].interpolate())
data['PS'] = data.groupby(['PS']).apply(lambda x: x[['PS']].interpolate())
data['T2M'] = data.groupby(['T2M']).apply(lambda x: x[['T2M']].interpolate())
data['WS50M'] = data.groupby(['WS50M']).apply(lambda x: x[['WS50M']].interpolate())
data['ALLSKY_SFC_LW_DWN'] = data.groupby(['ALLSKY_SFC_LW_DWN']).apply(lambda x: x[['ALLSKY_SFC_LW_DWN']].interpolate())

data = data.set_index(keys='time')
robustscaler = preprocessing.RobustScaler()
for i in range(data.shape[1]):
    x_scaled = robustscaler.fit_transform(data.iloc[:,i].values.reshape(-1,1))
    data.iloc[:,i] = x_scaled
year = np.array(data.loc['1985':'1986'])

#train with miniSOM library
s2 = som.MiniSom(x=15, y=3, input_len=year.shape[1], sigma=1.0, learning_rate=0.8, random_seed=8)
s2.train_random(year, 10000, verbose=False)

In [ ]:
#np.save('_weights.npy',s2._weights)
np.array(data.loc['1985':'1986'])

In [ ]:
#visualization
pysomviz2 = PySOMVis(weights=s2._weights, input_data=np.array(data.loc['1985':'2020']))
pysomviz2._mainview

In [ ]:
from SOMToolBox_Parse import SOMToolBox_Parse
from pysomvis import PySOMVis
from sklearn import datasets
import holoviews as hv
import numpy as np
from typing import Tuple, List
from visualizations.graph import Graph

iris = datasets.load_iris()

iris_data = SOMToolBox_Parse('datasets\\iris\\iris.vec').read_weight_file()
iris_weights = SOMToolBox_Parse('datasets\\iris\\iris.wgt.gz').read_weight_file()

#visualization
pysomviz3 = PySOMVis(weights=iris_weights['arr'], m=iris_weights['ydim'], n=iris_weights['xdim'],
                                                  dimension=iris_weights['vec_dim'], input_data=iris.data)
pysomviz3._mainview